In [6]:
import scipy.stats as stats
import scipy.special as special
import numpy as np
import scipy.optimize as opt
import scipy.stats as st
import scipy.stats
import numpy as np
from scipy import stats
from scipy.stats import pearsonr, spearmanr, iqr, median_abs_deviation, beta, norm, nct, t, f, ncf, chi2,  median_test, hmean, wilcoxon, hypergeom, rankdata, mannwhitneyu, binom, fisher_exact, barnard_exact, chi2_contingency
from scipy.special import hyp2f1
from numpy import around, array2string
import numpy as np
import numpy as geek
import math
import random
import itertools
import statistics
import researchpy as rp
import pingouin as pg
from pingouin import cochran
import pandas as pd
import itertools as it
from collections import Counter
import scipy.stats as stats
import scipy.optimize as optimize
from scipy.optimize import newton, root_scalar
from scipy.stats.contingency import odds_ratio, relative_risk
import rpy2.robjects as robjects
from statsmodels.stats.contingency_tables import mcnemar, cochrans_q
from statsmodels.stats.proportion import proportion_confint, confint_proportions_2indep
from itertools import product

a = 20
b = 10
c = 5
d = 14
confidence_level = 0.95

def calculate_p_value_from_chi_score(chi_score, df):
    p_value = st.chi2.sf((abs(chi_score)), df)
    return min(float(p_value), 0.99999)


def Binary_2x2_measures(a,b,c,d): 
    sample_size = a+b+c+d
    Corrected_levels_Number = 2 - 1/(sample_size-1)
    
    
    # Effect Sizes
    Odds_Ratio = a*d/ (b*c)
    Tetrachoric_Correlation = np.cos(3.14159265359 / (1 + np.sqrt(Odds_Ratio))) 
    Tetrachoric_Basic_Approximation = (Odds_Ratio**0.74 - 1) / (Odds_Ratio**0.74 + 1)
    phi = (a*d - b*c) / np.sqrt((a+b)*(c+d)*(a+c)*(b+d))
    Chambers_R = (((Odds_Ratio + 1)/(Odds_Ratio - 1))  -  ((2*Odds_Ratio * np.log(Odds_Ratio)) / (Odds_Ratio - 1)**2))
    Cramer = phi
    Bias_Corrected_Cramer = np.sqrt((phi**2/(Corrected_levels_Number-1)) ) # CORRECTION IS FROM BERGSMA 2013
    
    # Maximum Corrected Phi (See richard liu 1980)
    max_phi1 = np.sqrt( (  (a+b)/(c+d)) * ((d+b) / (c+a)))
    max_phi2 = np.sqrt( (  (a+c)/(b+d)) * ((a+b) / (c+d)))  
    max_phi = max_phi1 if c + d > b + a else max_phi2
    Max_Corrected_Phi = phi/max_phi

    # Significance_of_phi and max corrected phi
    chi_square_phi = phi**2*sample_size
    p_value = calculate_p_value_from_chi_score(chi_square_phi, 1)
    chi_square_max_corrected_phi = Max_Corrected_Phi**2*sample_size
    p_value_max_Corrected_phi = calculate_p_value_from_chi_score(chi_square_max_corrected_phi, 1)

    # Variance of phi (from Bishop et al's., book) (approximation for large samples)
    p1_plus = (a +b)/sample_size # Marginal_Probability_Row1
    p2_plus = (c +d)/sample_size # Marginal_Probability_Row2
    pplus_1 = (a +c)/sample_size # Marginal_Probability_Coloumn1
    pplus_2 = (b +d)/sample_size # Marginal_Probability_Coloumn2
    Probabilities_Product = (p1_plus*p2_plus*pplus_1*pplus_2)
    term1 =  phi + 0.5 * phi**3
    term2 = ((p1_plus - p2_plus) * (pplus_1 - pplus_2)) / np.sqrt(Probabilities_Product)
    term3 = 0.75*phi**2*(((p1_plus - p2_plus)**2 / (p1_plus * p2_plus)) + ((pplus_1 - pplus_2)**2 / (pplus_1 * pplus_2)))
    Variance = (1/sample_size) * (1 - phi**2 + term1 * term2 - term3)

    # Corrected Tetrachoric Correlaiton and Its Variance based on the odds ratio variance (Bonnet, 2005)
    
    # Preaper the Correction 
    r1 = (a + b + 1) / (sample_size + 2)
    r2 = (c + d + 1) / (sample_size + 2)
    c1 = (a + b + 1) / (sample_size + 2)
    c2 = (c + d + 1) / (sample_size + 2)
    Minimal_Probability = min(c1, c2, r1, r2)
    Correction = (1 - abs(r1 - c1) / 5 - (0.5 - Minimal_Probability)**2) / 2
    
    Corrected_Odds_Ratio = (a + 0.5)*(b + 0.5)/((c + 0.5)*(d + 0.5))
    Corrected_Tetrachoric_Correlation = np.cos(3.14159/ (1 + Corrected_Odds_Ratio**Correction))

    Standard_Error_Odds_Ratio = np.sqrt(1/a + 1/b + 1/c + 1/d)
    Standard_Error_Odds_Ratio_Corrected = np.sqrt(1 / (a + 0.5) + 1 / (b + 0.5) + 1 / (c + 0.5) + 1 / (d + 0.5))

    K = (3.14159 * 0.5 * Odds_Ratio**0.5) * np.sin(3.14159 / (1 + Odds_Ratio**0.5)) / (1 + Odds_Ratio**0.5)**2
    Corrected_K = (3.14159 * Correction * Corrected_Odds_Ratio**Correction) * np.sin(3.14159/(1 + Corrected_Odds_Ratio**Correction)) / (1 + Corrected_Odds_Ratio**Correction)**2
    
    Standard_Error_Tetrachoric = K * Standard_Error_Odds_Ratio_Corrected
    Standard_Error_Tetrachoric_Corrected = Corrected_K * Standard_Error_Odds_Ratio_Corrected

    #Wallis Swing and percentage Swing Measures
    Wallis_Swing_d_coloumns_indpendnt =  a/(a+b) - c/(c+d)
    Wallis_Perecentage_Swing_coloumns_independent =  Wallis_Swing_d_coloumns_indpendnt /  (a/(a+b))
    Estimate = (a+c)/sample_size
    Standard_Error_coloumns_independent = np.sqrt(( (Estimate*(1-Estimate)) * (1/(a+b)+(1/(c+d)))))
    z_crit = st.norm.ppf(confidence_level + ((1 - confidence_level) / 2))
    Lower_CI_swing_d_Wallis = Wallis_Swing_d_coloumns_indpendnt - Standard_Error_coloumns_independent*z_crit
    Upper_CI_swing_d_Wallis = Wallis_Swing_d_coloumns_indpendnt + Standard_Error_coloumns_independent*z_crit
    Lower_CI_Percentage_swing_d_Wallis = Lower_CI_swing_d_Wallis / (a/(a+b))
    Upper_CI_Percentage_swing_d_Wallis = Upper_CI_swing_d_Wallis / (a/(a+b))


    results = {}

    results["Odds Ratio"] = round(Odds_Ratio, 4)
    results["Chambers r"] = round(Chambers_R, 4)
    
    # Tetrachoric Correlation
    results["Tetrachoric Correlation"] = round(Tetrachoric_Correlation, 4)
    results["Tetrachoric Approximation"] = round(Tetrachoric_Basic_Approximation, 4)
    results["Tetrachoric Corrected"] = round(Corrected_Tetrachoric_Correlation, 4)
    results["Standard_Error_Tetrachoric"] = round(Standard_Error_Tetrachoric, 4)
    results["Standard_Error_Tetrachoric_Corrected"] = round(Standard_Error_Tetrachoric_Corrected, 4)

    # Phi and max phi
    results["Phi"] = round(phi, 4)
    results["Phi chi square statistic"] = round(chi_square_phi, 4)
    results["Phi p_value"] = round(p_value, 4)
    results["maxphi"] = round(max_phi, 4)
    results["Max_Corrected_Phi)"] = round(Max_Corrected_Phi, 4)
    results["Phi max_Corrected_p_value"] = round(p_value_max_Corrected_phi, 4)
    results["Phi Variance"] = round(Variance, 7)
    results["chi square phi"] = round(chi_square_phi, 5)
    results["chi square max corrected phi"] = round(chi_square_max_corrected_phi, 5)

    # Other Basic Effect Sizes
    results["Cramer's V)"] = round(phi, 4)
    results["Bias Corrected Cramer)"] = round(Bias_Corrected_Cramer, 4)
    results["chi square phi"] = round(chi_square_phi, 5)


    # Wallis Measures of effect size
    results["Wallis Swing D (coloumns independent)"] = round(Wallis_Swing_d_coloumns_indpendnt, 4)
    results["Wallis Swing D Standard Error"] = round(Standard_Error_coloumns_independent, 5)
    results["Wallis Percentage Swing (coloumns independent)"] = round(Wallis_Perecentage_Swing_coloumns_independent, 4)
    results["CI_swing_d_Wallis"] = f"({round(Lower_CI_swing_d_Wallis, 4)}, {round(Upper_CI_swing_d_Wallis, 4)})"
    results["CI_Percentage_swing_d_Wallis"] = f"({round(Lower_CI_Percentage_swing_d_Wallis, 4)}, {round(Upper_CI_Percentage_swing_d_Wallis, 4)})"


    #result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    #return result_str

    return results


Binary_2x2_measures(a,b,c,d)






{'Odds Ratio': 5.6,
 'Chambers r': 0.5229,
 'Tetrachoric Correlation': 0.5953,
 'Tetrachoric Approximation': 0.5631,
 'Tetrachoric Corrected': 0.3689,
 'Standard_Error_Tetrachoric': 0.1656,
 'Standard_Error_Tetrachoric_Corrected': 0.2136,
 'Phi': 0.3933,
 'Phi chi square statistic': 7.5792,
 'Phi p_value': 0.0059,
 'maxphi': 1.2825,
 'Max_Corrected_Phi)': 0.3067,
 'Phi max_Corrected_p_value': 0.0318,
 'Phi Variance': 0.0169076,
 'chi square phi': 7.57924,
 'chi square max corrected phi': 4.60818,
 "Cramer's V)": 0.3933,
 'Bias Corrected Cramer)': 0.3975,
 'Wallis Swing D (coloumns independent)': 0.4035,
 'Wallis Swing D Standard Error': 0.14657,
 'Wallis Percentage Swing (coloumns independent)': 0.6053,
 'CI_swing_d_Wallis': '(0.1162, 0.6908)',
 'CI_Percentage_swing_d_Wallis': '(0.1744, 1.0362)'}